In [360]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [361]:
#read in files
#already been split into training people and testing groups
#made from impute2

testing_SNPs = pd.read_csv('testing.txt', delimiter=' ', header = None)
training_SNPs = pd.read_csv('training.txt', delimiter = ' ', header = None)
#map22 = pd.read_csv('map22.txt', delimiter = '\t', header = None)
map22_2 = pd.read_csv('chr22.txt', delimiter = ' ', header = None)

In [362]:
#read in map
#find how many SNPs in our training/testing SNPs (they're the same...) are also in our map

print(training_SNPs.shape) #we only have 250 people in our training data
#print(map22[:6][0]) #first col is physical position, second is recombination rate with the next SNP, \
#third is genetic map in cM

training_SNPs_arr = training_SNPs[:][2].to_numpy()

#map22_arr_pos = map22.loc[:][0].to_numpy()

#this map has more data in it
chr22_arr = map22_2.loc[:8000][1].to_numpy() #hard coding to make file smaller
chr22_cM = map22_2.loc[:8000][2].to_numpy()

#turns out there are a few duplicates in my chr22 map
uniques = np.unique(chr22_arr)
print(len(uniques))


#find indices in training_SNPs_arr where we have a cM distance
indices_in_SNPs = np.where(np.in1d(training_SNPs_arr,chr22_arr))
indices_in_map = np.where(np.in1d(chr22_arr, training_SNPs_arr))

print(len(indices_in_SNPs[0]))
print(len(indices_in_map[0]))

idx_map = indices_in_map[0]
idx_SNP = indices_in_SNPs[0]

print(idx_map)


(5000, 505)
7993
2486
2488
[   5    8   14 ... 7250 7251 7260]


In [363]:
#make a 2 column matrix
#first column = SNP position
#second column = cM distance
#if no cM distance, put -1 for now

cM_map = np.zeros((SNPs,2))
for SNP in range(SNPs):
    #first column = SNP position
    cM_map[SNP, 0] = training_SNPs_arr[SNP]
    
    #check if the SNP is found by indices_in_SNPs
    if SNP in idx_SNP:
        #ind = index for which indices_in_SNPs[index] == SNP
        ind = np.where(idx_SNP == SNP)
        #print(type(idx_map[ind][0]))
        cM_map[SNP, 1] = chr22_cM[idx_map[ind][0]]
    else:
        cM_map[SNP, 1] = -1
        
#going to initialize this first value to a cM value of 0
print(cM_map[0][1])
cM_map[0][1] = 0
print(cM_map[:20])


-1.0
[[ 1.60506070e+07  0.00000000e+00]
 [ 1.60508400e+07 -1.00000000e+00]
 [ 1.60508470e+07 -1.00000000e+00]
 [ 1.60512490e+07  0.00000000e+00]
 [ 1.60514770e+07  1.05258065e-03]
 [ 1.60520800e+07  5.93493548e-03]
 [ 1.60524630e+07 -1.00000000e+00]
 [ 1.60526840e+07  1.08277256e-02]
 [ 1.60528370e+07 -1.00000000e+00]
 [ 1.60529620e+07 -1.00000000e+00]
 [ 1.60529860e+07 -1.00000000e+00]
 [ 1.60531390e+07 -1.00000000e+00]
 [ 1.60532490e+07 -1.00000000e+00]
 [ 1.60532540e+07 -1.00000000e+00]
 [ 1.60534440e+07  1.70082028e-02]
 [ 1.60536590e+07  1.87560000e-02]
 [ 1.60537300e+07  1.93333232e-02]
 [ 1.60537910e+07  1.98292848e-02]
 [ 1.60538620e+07  2.04065037e-02]
 [ 1.60538630e+07 -1.00000000e+00]]


In [364]:
#need to infer missing cM values
#going to just find average value (is this a valid assumption?)


for SNP in range(20):
    counter = 1
    if cM_map[SNP][1] < 0:
        i = 0
        prev_known_SNP = cM_map[SNP-1][1]
        while (cM_map[SNP + i][1] < 0):
            counter +=1 
            i += 1
        next_known_SNP = cM_map[SNP + i][1]
        print("i", i, "counter", counter)
        print("prev", prev_known_SNP, "next", next_known_SNP)
        cM_map[SNP][1] = (next_known_SNP - prev_known_SNP)/counter + prev_known_SNP
        
print(cM_map[:20])


i 2 counter 3
prev 0.0 next 0.0
i 1 counter 2
prev 0.0 next 0.0
i 1 counter 2
prev 0.00593493548387 next 0.0108277256461
i 6 counter 7
prev 0.0108277256461 next 0.017008202783299998
i 5 counter 6
prev 0.011710650951414285 next 0.017008202783299998
i 4 counter 5
prev 0.012593576256728571 next 0.017008202783299998
i 3 counter 4
prev 0.013476501562042857 next 0.017008202783299998
i 2 counter 3
prev 0.014359426867357142 next 0.017008202783299998
i 1 counter 2
prev 0.015242352172671428 next 0.017008202783299998
i 4 counter 5
prev 0.020406503664900003 next 0.026983546596900002
[[1.60506070e+07 0.00000000e+00]
 [1.60508400e+07 0.00000000e+00]
 [1.60508470e+07 0.00000000e+00]
 [1.60512490e+07 0.00000000e+00]
 [1.60514770e+07 1.05258065e-03]
 [1.60520800e+07 5.93493548e-03]
 [1.60524630e+07 8.38133056e-03]
 [1.60526840e+07 1.08277256e-02]
 [1.60528370e+07 1.17106510e-02]
 [1.60529620e+07 1.25935763e-02]
 [1.60529860e+07 1.34765016e-02]
 [1.60531390e+07 1.43594269e-02]
 [1.60532490e+07 1.5242352

In [51]:
#convert testing file into 3 part prob dist file
#for each SNP, each person, who is 100% AA, AB, or BB, which is written as 00, 01, or 11, gets 
#turned into one number for AA, one number for AB, one number for BB (so 1, 0,0 as AA, 0, 1, 0 as
#AB, and 0,0,1 for BB)


SNPs = int(testing_SNPs.shape[0])
people = int((testing_SNPs.shape[1]-5) / 2.0)

testing_dist = np.zeros((SNPs, people * 3))
maj_homozygous_test = np.zeros((SNPs, people))
heterozygous_test = np.zeros((SNPs, people))
min_homozygous_test = np.zeros((SNPs, people))

#need to come back to this...
for SNP in range(SNPs):
    for person in range(people):
        if testing_SNPs.loc[SNP, 5 + person*2] == 0 & testing_SNPs.loc[SNP, 6 + person*2] == 0:
            testing_dist[SNP, 3*person] = 1
            testing_dist[SNP, 3*person + 1] = 0
            testing_dist[SNP, 3*person + 2] = 0
            maj_homozygous_test[SNP, person] = 1
        elif testing_SNPs.loc[SNP, 5 + person*2] == 1 & testing_SNPs.loc[SNP, 6 + person*2] == 0:
            testing_dist[SNP, 3*person] = 0
            testing_dist[SNP, 3*person + 1] = 1
            testing_dist[SNP, 3*person + 2] = 0
            heterozygous_test[SNP, person] = 1
        elif testing_SNPs.loc[SNP, 5 + person*2] == 0 & testing_SNPs.loc[SNP, 6 + person*2] == 1:
            testing_dist[SNP, 3*person] = 0
            testing_dist[SNP, 3*person + 1] = 1
            testing_dist[SNP, 3*person + 2] = 0
            heterozygous_test[SNP, person] = 1
        elif testing_SNPs.loc[SNP, 5 + person*2] == 1 & testing_SNPs.loc[SNP, 6 + person*2] == 1:
            testing_dist[SNP, 3*person] = 0
            testing_dist[SNP, 3*person + 1] = 0
            testing_dist[SNP, 3*person + 2] = 1
            min_homozygous_test[SNP, person] = 1
            


people 253


In [68]:
print(training_SNPs.shape)
training_SNPs = training_SNPs

(5000, 505)


In [70]:
#convert training file into 3 part prob dist file
#for each SNP, each person, who is 100% AA, AB, or BB, which is written as 00, 01, or 11, gets 
#turned into one number for AA, one number for AB, one number for BB (so 1, 0,0 as AA, 0, 1, 0 as
#AB, and 0,0,1 for BB)

SNPs = int(training_SNPs.shape[0])
people_training = int((training_SNPs.shape[1]-5) / 2.0)

training_dist = np.zeros((SNPs, people_training * 3))
maj_homozygous_train = np.zeros((SNPs, people_training))
heterozygous_train = np.zeros((SNPs, people_training))
min_homozygous_train = np.zeros((SNPs, people_training))

#need to come back to this...
for SNP in range(SNPs):
    for person in range(people_training):
        if training_SNPs.loc[SNP, 5 + person*2] == 0 & training_SNPs.loc[SNP, 6 + person*2] == 0:
            training_dist[SNP, 3*person] = 1
            training_dist[SNP, 3*person + 1] = 0
            training_dist[SNP, 3*person + 2] = 0
            maj_homozygous_train[SNP, person] = 1
        elif training_SNPs.loc[SNP, 5 + person*2] == 1 & training_SNPs.loc[SNP, 6 + person*2] == 0:
            training_dist[SNP, 3*person] = 0
            training_dist[SNP, 3*person + 1] = 1
            training_dist[SNP, 3*person + 2] = 0
            heterozygous_train[SNP, person] = 1
        elif training_SNPs.loc[SNP, 5 + person*2] == 0 & training_SNPs.loc[SNP, 6 + person*2] == 1:
            training_dist[SNP, 3*person] = 0
            training_dist[SNP, 3*person + 1] = 1
            training_dist[SNP, 3*person + 2] = 0
            heterozygous_train[SNP, person] = 1
        elif training_SNPs.loc[SNP, 5 + person*2] == 1 & training_SNPs.loc[SNP, 6 + person*2] == 1:
            training_dist[SNP, 3*person] = 0
            training_dist[SNP, 3*person + 1] = 0
            training_dist[SNP, 3*person + 2] = 1
            min_homozygous_train[SNP, person] = 1


In [107]:
#Transpose the matrices so they are people # rows and SNP # columns

maj_homozygous_train.transpose()
heterozygous_train.transpose()
min_homozygous_train.transpose()

#generate training data + labels for very first SNP
idx_snp = 100
offset = 20

label_maj_homo = maj_homozygous_train[:, idx_snp]
training_maj_homo = np.hstack((maj_homozygous_train[:, (idx_snp - 20):idx_snp], maj_homozygous_train[:, (idx_snp + 1):(idx_snp + 1 + offset)]))

snp100 = RandomForestClassifier(n_estimators = 200, max_depth=4, class_weight = "balanced")
snp100.fit(training_maj_homo, label_maj_homo)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [108]:
maj_homozygous_test.transpose()
heterozygous_test.transpose()
min_homozygous_test.transpose()

label_maj_homo_test = maj_homozygous_test[:, idx_snp]
testing_maj_homo = np.hstack((maj_homozygous_test[:, (idx_snp - 20):idx_snp], maj_homozygous_train[:, (idx_snp + 1):(idx_snp + 1 + offset)]))
predict_label_maj_100 = snp100.predict(testing_maj_homo)

print(np.sum(predict_label_maj_100 == label_maj_homo_test)/len(label_maj_homo_test))
print("predicted", np.sum(predict_label_maj_100))
print("actual values", np.sum(label_maj_homo_test))

0.784
predicted 823.0
actual values 601.0


In [9]:
print(type(min_homozygous))
print(min_homozygous.shape)

<class 'numpy.ndarray'>
(5000, 253)


In [43]:
#calculate indices for imputing
'''
thin_var = 0.5
np.random.seed(0)

idx_imputing = np.random.choice(int(len(maj_homozygous)), int(len(maj_homozygous) * thin_var), replace = False)
sorted_idx = np.sort(idx_imputing)

print(people)
print(maj_homozygous.shape)'''

250
(5000, 253)


In [60]:
#actually remove indices of interest

maj_homoz_toImpute = np.zeros((SNPs, people))
hetero_toImpute = np.zeros((SNPs, people))
min_homoz_toImpute = np.zeros((SNPs, people))

#not sure if there's any point to 
maj_homoz_labels = np.zeros((int(len(maj_homozygous) * thin_var), 2))

for SNP in range(SNPs):
    if SNP in sorted_idx:
        maj_homoz_toImpute[SNP] = maj_homozygous[SNP]
    else:
        maj_homoz_toImpute[SNP] = 2 #2 would mean it's to be imputed
    


5000


In [ ]:
#create "labels" vector for training
#starting with major homozygous

In [61]:
#the RF model will produce the "labels" vector (testing)
#starting with major homozygous 
#need to create the data matrix

testing_data = np.zeros((int(len(maj_homozygous) * thin_var) * people , 20))
counter = 1

for SNP in range(SNPs):
    if SNP in sorted_idx:
        #do nothing
    else:
        if SNP < 10 or SNP > (SNPs - 11): 
        else:
            for person in people:
                for i in range(10):
                    testing_data[person*counter, i] = 


#for SNP in range(int(len(maj_homozygous) * thin_var)): #2500
    

2500
